In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df_menu = pd.read_excel('./part.xlsx',
                        header=0,
                        dtype=str).fillna('')

list_part_number = df_menu['Part_Number'].tolist()
list_brand = df_menu['Brand'].tolist()
list_url = ['https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=' + str(i) + '&brand=' + str(j) for i, j in zip(list_part_number, list_brand)]

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests
import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for part_number, url in zip(list_part_number, list_url):
    work.put_nowait((part_number, url))

def crawler():
    global df
    
    while not work.empty():
        part_number, url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                resp = requests.get(url, proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                info = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                src = info[0]['image_BG_Url']
                
                # = = = = = = = = = = = = = = =
                
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Part_Number': part_number,
                                         'Src': src}])
                
                status = 'ok'

                break
                
            except:
                time.sleep(3)
                continue
                
        # = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_Number': part_number}])

        # = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
df.to_excel('./src_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：467

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS108&brand=STD  <->  [ok] - 剩余数量：447
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APK103&brand=STD  <->  [ok] - 剩余数量：446
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS101&brand=STD  <->  [ok] - 剩余数量：445
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS104&brand=STD  <->  [ok] - 剩余数量：444
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS103&brand=STD  <->  [ok] - 剩余数量：443
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS102&brand=STD  <->  [ok] - 剩余数量：442
https://ecatalog.smpcorp.com/V2/STD/a

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS162&brand=STI  <->  [ok] - 剩余数量：391
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS164&brand=STI  <->  [ok] - 剩余数量：390
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS159&brand=STI  <->  [ok] - 剩余数量：389
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS158&brand=STI  <->  [ok] - 剩余数量：388
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS166&brand=STI  <->  [ok] - 剩余数量：387
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS171&brand=STI  <->  [ok] - 剩余数量：386
https://ecatalog.smpcorp.com/V2/STD/api/image/

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS224&brand=STD  <->  [ok] - 剩余数量：339
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS240&brand=STI  <->  [ok] - 剩余数量：338
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS237&brand=STI  <->  [ok] - 剩余数量：337
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS232&brand=STI  <->  [ok] - 剩余数量：336
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS241&brand=STI  <->  [ok] - 剩余数量：335
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS229&brand=STI  <->  [ok] - 剩余数量：334
https://ecatalog.smpcorp.com/V2/STD/api/image/

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS307&brand=STI  <->  [ok] - 剩余数量：284
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS297&brand=STD  <->  [ok] - 剩余数量：283
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS293&brand=STD  <->  [ok] - 剩余数量：282
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS296&brand=STI  <->  [ok] - 剩余数量：281
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS306&brand=STI  <->  [ok] - 剩余数量：280
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS319&brand=STI  <->  [ok] - 剩余数量：279
https://ecatalog.smpcorp.com/V2/STD/api/image/

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS370&brand=STI  <->  [ok] - 剩余数量：230
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS373&brand=STD  <->  [ok] - 剩余数量：229
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS321&brand=STI  <->  [ok] - 剩余数量：228
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS372&brand=STD  <->  [ok] - 剩余数量：227
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS377&brand=STI  <->  [ok] - 剩余数量：226
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS376&brand=STI  <->  [ok] - 剩余数量：225
https://ecatalog.smpcorp.com/V2/STD/api/image/

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS427&brand=STI  <->  [ok] - 剩余数量：176
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS432&brand=STI  <->  [ok] - 剩余数量：175
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS434&brand=STI  <->  [ok] - 剩余数量：174
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS431&brand=STI  <->  [ok] - 剩余数量：173
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS442&brand=STI  <->  [ok] - 剩余数量：172
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS436&brand=STI  <->  [ok] - 剩余数量：171
https://ecatalog.smpcorp.com/V2/STD/api/image/

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS519&brand=STI  <->  [ok] - 剩余数量：121
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS520&brand=STI  <->  [ok] - 剩余数量：120
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS513&brand=STI  <->  [ok] - 剩余数量：119
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS510&brand=STI  <->  [ok] - 剩余数量：118
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS523&brand=STI  <->  [ok] - 剩余数量：117
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS526&brand=STI  <->  [ok] - 剩余数量：116
https://ecatalog.smpcorp.com/V2/STD/api/image/

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS596&brand=STI  <->  [ok] - 剩余数量：69
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS594&brand=STD  <->  [ok] - 剩余数量：68
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS598&brand=STI  <->  [ok] - 剩余数量：67
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS597&brand=STI  <->  [ok] - 剩余数量：66
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS605&brand=STI  <->  [ok] - 剩余数量：65
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS603&brand=STD  <->  [ok] - 剩余数量：64
https://ecatalog.smpcorp.com/V2/STD/api/image/getall

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS658&brand=STI  <->  [ok] - 剩余数量：16
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS657&brand=STI  <->  [ok] - 剩余数量：15
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS666&brand=STD  <->  [ok] - 剩余数量：14
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS669&brand=STI  <->  [ok] - 剩余数量：13
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS644&brand=STI  <->  [ok] - 剩余数量：12
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=APS629&brand=STD  <->  [ok] - 剩余数量：11
https://ecatalog.smpcorp.com/V2/STD/api/image/getall

D:\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
